In [2]:
%load_ext autoreload
%autoreload 1
import os
import sys
sys.path.append("../..")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric

import numpy as np
from tqdm.auto import tqdm

from ntupleReaders.clue_ntuple_reader import ClueNtupleReader
%aimport ml.regression.rechits
from ml.regression.rechits import *

In [3]:
reader_sim = ClueNtupleReader("v40", "cmssw", "sim_proton_v46_patchMIP")
reader_data = ClueNtupleReader(reader_sim.version, reader_sim.clueParams, "data")
#data_list = torch.load(os.path.join(reader_sim.pathToFolder, "rechitsGeometric.pt"))

In [4]:
dataset = TracksterPropertiesDataset(reader_sim)

In [5]:
dataset[5]

Data(x=[541, 4], beamEnergy=[1], trueBeamEnergy=[1])

In [6]:
totalev = len(dataset)
ntrain = int(0.8*totalev)
shuffled_dataset = dataset.shuffle()
ntrainbatch = 200
ntestbatch = 100
trainloader = torch_geometric.loader.DataLoader(shuffled_dataset[:ntrain], batch_size=ntrainbatch)
testloader = torch_geometric.loader.DataLoader(shuffled_dataset[ntrain:], batch_size=ntestbatch)
#batch_size = ntrainbatch
epoch_size = len(shuffled_dataset[:ntrain])
print("epoch size,batch_size:",epoch_size,ntrainbatch)

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

epoch size,batch_size: 287740 200


2023-06-08 11:44:16.570437: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 11:44:18.045881: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [7]:
from ml.dynamic_reduction_network import DynamicReductionNetwork
from ml.cyclic_lr import CyclicLRWithRestarts

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drn = DynamicReductionNetwork(input_dim=4,hidden_dim=20,k=10,output_dim=1,norm=torch.tensor([ 1., 1., 1., 1.]))
        
    def forward(self, data):
        logits = self.drn(data)
        #return F.softplus(logits)
        return logits
device = torch.device('cuda:1')#('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
#optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)
scheduler = CyclicLRWithRestarts(optimizer, ntrainbatch, epoch_size, restart_period=80, t_mult=1.2, policy="cosine")
#criterion = torch.nn.MSELoss()

#lossmse = nn.MSELoss()
def resoloss2(output,truth):
    """ Computes MSE loss between output and truth """
    batch_size = output.size()[0]
    mse = F.mse_loss(output, truth, reduction='mean')
    #mse = torch.sum((output-truth)**2/truth)/batch_size
    #mse = torch.sum(((output-truth)/truth)**2)/batch_size
    #mse = torch.mean(torch.abs((output - truth)))
    
    #res = 
    return (mse)

#model.train()
def train(epoch):
    model.train()
    torch.cuda.empty_cache()
    scheduler.step()
    loss = []
    for data in tqdm(trainloader):
            data = data.to(device)
            optimizer.zero_grad()
            result = model(data)
            lossc = resoloss2(result, data.trueBeamEnergy)
            loss.append(lossc.item()) 
            lossc.backward()
            optimizer.step()
            scheduler.batch_step()
    print( 'batches for train:',len(loss)) 
    print('train loss:',np.mean(np.array(loss)))
    return np.mean(np.array(loss))

from scipy.stats import norm
import matplotlib.mlab as mlab
import scipy.stats as scs
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
#%matplotlib inline

def gaussian(x,  mean,a, sigma):
    return a * np.exp(-((x - mean)**2 / (2 * sigma**2)))

def evaluate(epoch):
        """"Evaluate the model"""
        model.zero_grad()
        torch.cuda.empty_cache()
        model.eval()
        pred = []
        true = []
        loss= []
        
        correct = 0
        predc = []
        truec = []
        for data in tqdm(testloader):
            data = data.to(device)        
            result = model(data)
            lossc = resoloss2(result, data.trueBeamEnergy)

            loss.append(lossc.item())

            for i in result:
                pred.append(i.detach().cpu())
                #predc.append(i.detach().cpu().argmax())
            for i in data.trueBeamEnergy.detach():
                true.append(i.detach().cpu())

        print('batches for test:', len(loss)) 
        print('test loss:',np.mean(np.array(loss)))
#        fracarr = np.array(frac

        
        preda = np.array(pred)
        """ Array of predictions (beam energy) """
        truea = np.array(true)
        """ Array of true beam energy """
        #preda = preda[:,2] ### added
        #truea = truea[:,1] ### added
        fracarr = (preda - truea)/truea
        """ Array of (prediction - truth)/truth """
        #print(preda,truea,fracarr)
        print('pred - true / true mean:',(np.mean(fracarr)))
        print('pred - true / true std:',(np.std(fracarr)))
        (mu, sigma) = norm.fit(fracarr)
        """ gaussian fit of (prediction - truth)/truth """
        print('mu,sig:',mu,sigma)
        

        #bin_heights, bin_borders, _ = plt.hist(fracarr,range=[-2,2], bins=100, label='histogram')
        #bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
        
        #### Histogram of (pred - true) / true 
        from matplotlib.pyplot import figure
        # fig = figure(figsize=(30, 20), dpi=40)
        # plt.rcParams['axes.labelsize'] = 36
        # plt.rcParams['axes.titlesize'] = 36
        # plt.hist(fracarr,bins=100,range=[-4,4])
        # plt.xticks(fontsize=16)
        # plt.yticks(fontsize=16)
        # plt.axvline(x=0.0,c='r')
        #plt.show()
        writer.add_histogram("Pred-truth / truth", fracarr, epoch)

        
        '''
        bins =  np.linspace(0,18,19)

        fig, axs = plt.subplots(6,3, figsize=(40, 40), facecolor='w', edgecolor='k')
        axs = axs.ravel()

        for i in tqdm(range (bins.size - 1)):
            predaa = preda[(truea >bins[i]) & (truea <bins[i+1]) ]
            trueaa = truea[(truea >bins[i]) & (truea <bins[i+1]) ]
            fracarr = (predaa - trueaa)/trueaa
            #if (fracarr < 0):
            axs[i].hist(fracarr,bins=100,range=[-4,4],)
            axs[i].set_xlabel('pred - true / true')
            axs[i].set_ylabel('counts')
            axs[i].set_title(str(bins[i])+" to "+str(bins[i+1]))
            #print (vals)
        if (epoch%5==0):
            plt.savefig('%s/mt-mp_frac_ep%d.png'%(plot_dir, epoch), bbox_inches='tight')
        plt.show()'''
        
        '''try:
            from matplotlib.pyplot import figure
            figure(figsize=(20, 10), dpi=30)
            plt.rcParams['axes.labelsize'] = 16
            plt.rcParams['axes.titlesize'] = 16
            popt, _ = curve_fit(gaussian, bin_centers, bin_heights, p0=[0., 100., 1.],bounds = ([-np.inf,0,0],[np.inf,np.inf,np.inf]))
            x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 100)
            plt.plot(x_interval_for_fit, gaussian(x_interval_for_fit, *popt), label='fit')
            plt.legend()


            plt.xlabel('pred - true / true')
            plt.ylabel('counts')
            #plt.yscale("log")
            #plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
            plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(popt[0], popt[2]))
            plt.grid(True)

            plt.show()

        except RuntimeError:
            print("Error - curve_fit failed")
            plt.xlabel('pred - true / true')
            plt.ylabel('counts')
            #plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
            plt.title('pred - true / true fit failed')
            #plt.yscale("log")
            plt.grid(True)

            plt.show()'''
        ### Scatter of true beam energy (x) vs predicted beam energy (y)
        from matplotlib.pyplot import figure
        fig = figure(figsize=(30, 30), dpi=40)
        #plt.rcParams['axes.labelsize'] = 36
        #plt.rcParams['axes.titlesize'] = 36
        #plt.hist2d(truea,preda,bins=200)
        plt.scatter(truea,preda,alpha=0.4)
        plt.plot([0,500], [0,500], 'k-')
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        #plt.xlim([-2, 18])
        #plt.ylim([-2, 20])
        plt.axvline(x=1.0,c='r')
        plt.axhline(y=1.0,c='r')
        #plt.show()
        writer.add_figure("Scatter", fig, epoch)
        #if (epoch%5 ==0):
        #    plt.savefig('%s/mpredVsmtrue_ep%d.png'%(plot_dir, epoch), bbox_inches='tight')
        #plt.show()
        return np.mean(np.array(loss))
        #del pred,true,loss,preda,truea,fracarr  #memtest


In [8]:
#from tqdm import notebook.tqdm as tqdm
#import tqdm.notebook.tqdm as tqdm
#checkpoint_dir = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_testtesttesttest_dyn2addlayer/'
#checkpoint_dir = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_add1ip1dyn_binresoloss/'
#"/home/llr/cms/sghosh/GNNECAL/model_nopho_deepernet/"
checkpoint_dir = "/grid_mnt/vol_home/llr/cms/cuisset/hgcal/testbeam18/clue3d-dev/src/Plotting/ml/regression/checkpoints"
plot_dir = "/grid_mnt/vol_home/llr/cms/cuisset/hgcal/testbeam18/clue3d-dev/src/Plotting/ml/regression/plots"
nepoch=500
os.makedirs(checkpoint_dir, exist_ok=True)
best_loss = 99999999
losst = []
""" Training loss for each epoch """
lossv = []
""" Validation loss for each epoch"""
epochs = []
for epoch in range(nepoch):
    print ('epoch:',epoch)
    losst.append(train(epoch))
    loss_epoch = evaluate(epoch)
    lossv.append(loss_epoch)
    epochs.append(epoch)
    checkpoint = {
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
    }
    
    checkpoint_file = 'model_epoch_%i.pth.tar' % ( epoch )
    torch.save(checkpoint,
                   os.path.join(checkpoint_dir,checkpoint_file ))
    if loss_epoch < best_loss:
        best_loss = loss_epoch
        print('new best test loss:',best_loss)
        torch.save(checkpoint,
                   os.path.join(checkpoint_dir,'model_checkpoint_best.pth.tar' ))
    
    writer.add_scalar("Loss/Training", losst[-1], epoch)
    writer.add_scalar("Loss/Testing", loss_epoch, epoch)

 

epoch: 0


  0%|          | 0/1439 [00:00<?, ?it/s]

/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/torch_geometric/utils/scatter.py:93: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(f"The usage of `scatter(reduce='{reduce}')` "


batches for train: 1439
train loss: 2938.9465175944124


  0%|          | 0/720 [00:00<?, ?it/s]

...

  0%|          | 0/1439 [00:00<?, ?it/s]

batches for train: 1439
train loss: 655.967830867383


  0%|          | 0/720 [00:00<?, ?it/s]

batches for test: 720
test loss: 405.1846091376411
pred - true / true mean: -0.08099406
pred - true / true std: 0.14379816
mu,sig: -0.08099406 0.14379816
epoch: 22


  0%|          | 0/1439 [00:00<?, ?it/s]